## 7.30 模板

在编译之前，模板参数被其值替换。在这一点上，模板类似于宏。
以下示例说明了函数参数和模板参数之间的区别：

```cpp
// Example 7.46
int Multiply (int x, int m) {
   return x * m;
}

template <int m>
int MultiplyBy (int x) {
   return x * m;
}

int a, b;
a = Multiply(10,8);
b = MultiplyBy<8>(10);
```

`a`和`b`都会得到计算结果：`10 * 8 = 80`。
不同之处在于`m`传递到函数的方式。
- 在简单函数中，`m`在运行时从调用者传递到被调用函数。
- 但是在模板函数中，`m`在编译时被其值替换，因此编译器会看到常量8而不是变量`m`。

使用模板参数，而不是函数参数，
- 优势是，避免了参数传输的开销。
- 缺点是，编译器需要为模板参数的每个不同值创建模板函数的新实例。

此示例中的`MultiplyBy`函数模板，如果使用许多不同因子作为模板参数来调用，则代码可能变得非常大。

在上面的示例中，模板函数比简单函数更快，这是因为编译器知道：可以通过使用移位操作来实现乘以2的幂。
即，`x * 8`由`x << 3`代替，速度更快。
在简单函数的情况下，编译器不知道`m`的值，因此除非可以内联函数，否则不能进行优化。
（在上面的例子中，编译器能够内联和优化两个函数，并简单地将80放入变量`a`和`b`。但在更复杂的情况下，它可能无法做到这样的优化）。

模板参数也可以是类型。
第38页上的示例显示了：如何使用相同的模板创建不同类型的数组。

模板是效率很高的，因为模板参数总是在编译时解析。
模板使源代码更复杂，但编译生成的代码并不会因此也复杂。
通常，就执行速度方面，在使用模板的没有成本。

如果模板参数完全相同，则将两个或多个模板实例会合并成一个。
如果模板参数不同，那么将为每组模板参数生成一个实例。
生成很多实例的模板会使编译的生成代码变大，并占用更多的缓存空间。

过度使用模板会使代码可读性差。
如果模板只有一个实例，那么你也可以使用`#define`，`const`或`typedef`，而不是模板参数。

模板可用于元编程，如第154页所述。

### 用模板实现多态

模板类可用于实现编译时多态，这比使用虚拟函数获得的运行时多态，效率更高。
下例首先示范了运行时多态性：

```cpp
// Example 7.47a. Runtime polymorphism with virtual functions
class CHello {
public:
   void NotPolymorphic(); // Non-polymorphic functions go here
   virtual void Disp(); // Virtual function
   void Hello() {
     cout << "Hello ";
     Disp(); // Call to virtual function
   }
};

class C1 : public CHello {
public:
   virtual void Disp() {
     cout << 1;
   }
};

class C2 : public CHello {
public:
   virtual void Disp() {
     cout << 2;
   }
};

void test () {
   C1 Object1; C2 Object2;
   CHello * p;
   p = &Object1;
   p->NotPolymorphic(); // Called directly
   p->Hello(); // Writes "Hello 1"
   p = &Object2;
   p->Hello(); // Writes "Hello 2"
}
```

如果编译器不知道`p`指向哪个类对象（参见第75页），只好在运行时来调度`C1::Disp()`或`C2::Disp()`。
当前的编译器还不太擅长优化掉`p`，改为内联对`Object1.Hello()`的调用，尽管未来的编译器可能会有这个能力。

如果在编译时知道对象是属于类C1还是C2，我们可以避免低效的虚函数调度过程。
**这可以通过使用的特殊技巧来完成。**该技巧已经用在活动模板库（ATL）和Windows模板库（WTL）中。
（译者注：这两个库都比较老，现在基本很少使用，但该技巧还是有效的。）

```cpp
// Example 7.47b. 使用模板实现编译时多态
// 把非多态函数放入祖父类中:
class CGrandParent {
public:
   void NotPolymorphic();
};

// 所有要调用多态函数的函数放入父类。
// 子类类名作为父类的模板参数：
template <typename MyChild>
class CParent : public CGrandParent {
public:
   void Hello() {
     cout << "Hello ";
     // call polymorphic child function:
     (static_cast<MyChild*>(this))->Disp();
   }
};

// 多个子类，每个子类实现一个版本的函数
class CChild1 : public CParent<CChild1> {
public:
   void Disp() {
     cout << 1;
   }
};

class CChild2 : public CParent<CChild2> {
public:
   void Disp() {
     cout << 2;
   }
};

void test () {
   CChild1 Object1; CChild2 Object2;
   CChild1 * p1;
   p1 = &Object1;
   p1->Hello(); // Writes "Hello 1"
   CChild2 * p2;
   p2 = &Object2;
   p2->Hello(); // Writes "Hello 2"
}
```


这里的`CParent`是一个模板类，它通过模板参数获取有关其子类的信息。
它可以将其“`this`”指针类型转换为指向其子类的指针，来调用其子类的多态成员函数。
仅仅当它具有正确的子类名作为模板参数时，这才是安全的。
换句话说，**你必须确保下面声明中，子类名称与和模板参数名称相同：**
```cpp
class CChild1 : public CParent<CChild1> {
```    

**继承顺序如下：**
- 第一代类（`CGrandParent`）包含所有非多态成员函数。
- 第二代类（`CParent<>`）包含需要调用多态函数的所有成员函数。
- 第三代类包含不同版本的多态函数。

第二代类通过模板参数获取有关第三代类的信息。

如果是已知对象的类，不会有时间浪费到运行时虚函数调度上。
该信息已经包含在具有不同类型的`p1`和`p2`中。
**缺点是`CParent::Hello()`有多个占用缓存空间的实例。**

示例7.47b中的语法无疑是非常复杂的。
避免虚函数调度来节省的这几个时钟周期，这个理由还不够充分，因为这样的设计带来了难以理解，难以维护的复杂代码。
如果编译器能够自动去虚拟化（devirtualization，参见第75页），依靠编译器进行优化的方法，肯定比使用复杂的模板方法更方便。